In [ ]:
!python3 -m pip install pip==24.0


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\vhgar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install numpy==1.23.5 scikit-learn==1.4.2  joblib pandas

  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
  Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl (10.6 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl (41.2 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install azureml-sdk

  Using cached backports.tempfile-1.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached msal-1.32.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached knack-0.12.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached azure_core-1.33.0-py3-none-any.whl.metadata (42 kB)
  Using cached pkginfo-1.12.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached argcomplete-3.6.2-py3-none-any.whl.metadata (16 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached paramiko-3.5.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached azure_mgmt_resource-23.3.0-py3-none-any.whl.metadata (41 kB)
  Using cached azure_mgmt_containerregistry-10.3.0-py3-none-any.whl.metadata (23 kB)
  Using cached azure_mgmt_keyvault-10.3.1-py3-none-any.whl.metadata (15 kB)
  Using cached azure_

## Carga de Susctiption ID


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()


server_ = os.getenv('AZURE_ID')

In [3]:
from azureml.core import Workspace

ws = Workspace.create(name="workspace_act3",
                      subscription_id = server_,
                      resource_group = "resource_group_act3",
                      create_resource_group=False,
                      location = "centralindia")


Deploying StorageAccount with name workspacstorage4964f8578.
Deploying AppInsights with name workspacinsights9d8349c9.
Deployed AppInsights with name workspacinsights9d8349c9. Took 11.92 seconds.
Deploying KeyVault with name workspackeyvault18c9f389.
Deployed StorageAccount with name workspacstorage4964f8578. Took 31.41 seconds.
Deploying Workspace with name workspace_act3.
Deployed KeyVault with name workspackeyvault18c9f389. Took 35.27 seconds.
Deployed Workspace with name workspace_act3. Took 41.4 seconds.


In [9]:
from azureml.core.model import Model

mname_scaler = "model_scaler"
registered_scaler = Model.register(model_path="../model/preprocessor_model.pkl",
                                  model_name=mname_scaler,
                                  workspace=ws)



Registering model model_scaler


In [10]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

virtual_env = Environment("env-act3")

virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['numpy==1.23.5','scikit-learn=1.4.2', 'joblib', 'pandas']
)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

In [8]:
from score import scorepy

print(scorepy)


import json 
import joblib
import pandas as pd
import numpy as np
import traceback
import sklearn
from azureml.core.model import Model

def embbed(d):
    try:
        required_columns = ['CustomerID', 'NameStyle', 'Title', 'FirstName', 'MiddleName',
       'LastName', 'Suffix', 'CompanyName', 'SalesPerson', 'EmailAddress',
       'Phone', 'PasswordHash', 'PasswordSalt', 'rowguid', 'ModifiedDate']
         
        missing_columns = [col for col in required_columns if col not in d.columns]
        if missing_columns:
            raise KeyError(f"Missing columns: {missing_columns}")

        return d[required_columns]

    except Exception as e:
        print(f"[ERROR] Unexpected error in embbed(): {e}")
        traceback.print_exc()
        return None

def init():
    global model, scaler

    try:
        print("[INFO] Loading model and scaler...")
        print(f"[INFO] NumPy version: {np.__version__}")
        print(f"[INFO] Scikit-learn version: {sklearn.__version__}")

        #

from utils.score import scorepy
